In [1]:
import itertools
import math
import numpy as np
import os
import random
import torch
import torch.nn.functional as F
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from itertools import chain
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import RandomHorizontalFlip
from transformers import CLIPTextModel, CLIPTokenizer, logging

# Setup
diffusion_model_id = 'runwayml/stable-diffusion-v1-5'
text_encoder_model_id = 'openai/clip-vit-large-patch14'
device = 'cuda'
seed = 1
torch.manual_seed(seed)
logging.set_verbosity_error()

# Textual inversion settings
property_name = 'bloodborne'  # Name of property to learn
property_type = 'style'  # Type of property to learn (object, style)
placeholder_token = f'<{property_name.replace("_", "-")}>'  # Token that represents new property
initializer_token = 'fantasy'  # Initial embedding for new property

# Hugging Face access token
token = ''
with open('hugging_face_token.txt', 'r') as secret:
    token = secret.readline().strip()

In [2]:
# Load model components

# Text Encoder + Tokenizer
tokenizer = CLIPTokenizer.from_pretrained(text_encoder_model_id)
text_encoder = CLIPTextModel.from_pretrained(text_encoder_model_id, torch_dtype=torch.float16)
text_encoder.to(device)

# Variational Autoencoder
vae = AutoencoderKL.from_pretrained(
    diffusion_model_id, subfolder='vae', torch_dtype=torch.float16,
    revision='fp16', use_auth_token=token)
vae.to(device)

# U-Net Model
u_net = UNet2DConditionModel.from_pretrained(
    diffusion_model_id, subfolder='unet', torch_dtype=torch.float16,
    revision='fp16', use_auth_token=token)
u_net.to(device)

# Noise Scheduler
noise_scheduler = DDPMScheduler.from_config(diffusion_model_id, subfolder='scheduler', use_auth_token=token)

# Freeze parameters for a model
def freeze_params(params):
    for param in params:
        param.requires_grad = False

# Freeze all pre-trained models except for token embeddings in the text encoder
freeze_params(vae.parameters())
freeze_params(u_net.parameters())
encoder_params_to_freeze = itertools.chain(
        text_encoder.text_model.encoder.parameters(),
        text_encoder.text_model.final_layer_norm.parameters(),
        text_encoder.text_model.embeddings.position_embedding.parameters(),
)
freeze_params(encoder_params_to_freeze)

In [3]:
# Setup tokenizer and text encoder

# Add the placeholder token in tokenizer
num_added_tokens = tokenizer.add_tokens(placeholder_token)

# Convert the initializer token and placeholder token to ids
token_ids = tokenizer.encode(initializer_token, add_special_tokens=False)
initializer_token_id = token_ids[0]
placeholder_token_id = tokenizer.convert_tokens_to_ids(placeholder_token)

# Resize the token embeddings as we are adding new special tokens to the tokenizer
text_encoder.resize_token_embeddings(len(tokenizer))

# Initialize the newly added placeholder token with the embeddings of the initializer token
token_embeds = text_encoder.get_input_embeddings().weight.data
token_embeds[placeholder_token_id] = token_embeds[initializer_token_id]

In [4]:
# Prompt templates

# Object
object_templates = [
    'a photo of a {}',
    'a rendering of a {}',
    'a cropped photo of the {}',
    'the photo of a {}',
    'a photo of a clean {}',
    'a photo of a dirty {}',
    'a dark photo of the {}',
    'a photo of my {}',
    'a photo of the cool {}',
    'a close-up photo of a {}',
    'a bright photo of the {}',
    'a cropped photo of a {}',
    'a photo of the {}',
    'a good photo of the {}',
    'a photo of one {}',
    'a close-up photo of the {}',
    'a rendition of the {}',
    'a photo of the clean {}',
    'a rendition of a {}',
    'a photo of a nice {}',
    'a good photo of a {}',
    'a photo of the nice {}',
    'a photo of the small {}',
    'a photo of the weird {}',
    'a photo of the large {}',
    'a photo of a cool {}',
    'a photo of a small {}',
]

# Style
style_templates = [
    'a painting in the style of {}',
    'a rendering in the style of {}',
    'a cropped painting in the style of {}',
    'the painting in the style of {}',
    'a clean painting in the style of {}',
    'a dirty painting in the style of {}',
    'a dark painting in the style of {}',
    'a picture in the style of {}',
    'a cool painting in the style of {}',
    'a close-up painting in the style of {}',
    'a bright painting in the style of {}',
    'a cropped painting in the style of {}',
    'a good painting in the style of {}',
    'a close-up painting in the style of {}',
    'a rendition in the style of {}',
    'a nice painting in the style of {}',
    'a small painting in the style of {}',
    'a weird painting in the style of {}',
    'a large painting in the style of {}',
]

In [5]:
# Dataset class
class TextualInversionDataset(Dataset):
    def __init__(
        self,
        data_root,
        learnable_property,
        placeholder_token,
        repeats=100,
        flip_p=0.5,
    ):
        self.data_root = data_root
        self.learnable_property = learnable_property
        self.placeholder_token = placeholder_token
        self.flip_p = flip_p
        self.flip_transform = RandomHorizontalFlip(p=self.flip_p)

        # Data settings
        self.image_paths = [os.path.join(self.data_root, file_path) for file_path in os.listdir(self.data_root)]
        self.num_images = len(self.image_paths)
        self._length = self.num_images * repeats
        self.templates = object_templates if property_type == 'object' else style_templates

    def __len__(self):
        return self._length

    def __getitem__(self, i):
        # Get and prepare image
        image = Image.open(self.image_paths[i % self.num_images])
        image = self.flip_transform(image)
        image = np.array(image).astype(np.uint8)
        image = (image / 127.5 - 1.0).astype(np.float16)

        # Get text prompt
        text = random.choice(self.templates).format(self.placeholder_token)

        # Create example
        example = {}
        example['input_prompt'] = text
        example['pixel_values'] = torch.from_numpy(image).permute(2, 0, 1).to(device)
        return example

In [6]:
# Auxiliary functions

# Encode input prompt
def encode_prompt(prompt):
    text_inputs = tokenizer(
        prompt, padding='max_length', max_length=tokenizer.model_max_length,
        truncation=True, return_tensors='pt')
    text_embeddings = text_encoder(text_inputs.input_ids.to(device))[0]
    return text_embeddings

In [7]:
# Save model to a pytorch file
def save_model(path_dir, filename):
    if not os.path.isdir(path_dir):
        os.makedirs(path_dir)
    learned_embeddings = text_encoder.get_input_embeddings().weight[placeholder_token_id]
    torch.save({placeholder_token: learned_embeddings.detach().cpu()}, os.path.join(path_dir, filename))

# Model training
def train_model(property, data_root, optimizer, max_train_steps=3000, batch_size=1):
    # Initialize dataset
    train_dataset = TextualInversionDataset(data_root, property_type, placeholder_token)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Set frozen models to eval
    vae.eval()
    u_net.eval()

    # Calculate number of training epochs
    num_train_epochs = math.ceil(max_train_steps / len(train_dataloader))

    # Training loop
    print('***** Running Training *****')
    print(f'  Num. Examples = {len(train_dataset)}')
    print(f'  Num. Epochs = {num_train_epochs}')
    for epoch in range(num_train_epochs):
        # Save current model
        save_model(f'saved_models/textual_inversion/{property}', f'{property}_{epoch}.pt')

        # Train for another epoch
        text_encoder.train()
        for step, batch in enumerate(train_dataloader):
            # Convert images to latent space
            latents = vae.encode(batch['pixel_values']).latent_dist.sample()
            latents *= 0.18215

            # Sample noise that we'll add to the latents
            noise = torch.randn(latents.shape, dtype=torch.float16).to(latents.device)
            bsz = latents.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device
            ).long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = encode_prompt(batch['input_prompt'])

            # Predict the noise residual
            noise_pred = u_net(noisy_latents, timesteps, encoder_hidden_states).sample

            # Backwards pass
            loss = F.mse_loss(noise_pred, noise, reduction='none').mean([1, 2, 3]).mean()
            loss.backward()

            # Zero out the gradients for all token embeddings except the placeholder token
            grads = text_encoder.get_input_embeddings().weight.grad
            index_grads_to_zero = torch.arange(len(tokenizer)) != placeholder_token_id
            grads.data[index_grads_to_zero, :] = grads.data[index_grads_to_zero, :].fill_(0)

            # Optimizer pass
            optimizer.step()
            optimizer.zero_grad()

            # Print logs
            current_loss = round(loss.detach().item(), 4)
            print(f'loss: {current_loss}, lr: {optimizer.param_groups[0]["lr"]}, '
                  f'epoch: {epoch + 1}/{num_train_epochs}, step: {step + 1}/{len(train_dataloader)}')
    print('***** Training Completed *****')

In [8]:
# Initialize optimizer
optimizer = torch.optim.AdamW(text_encoder.get_input_embeddings().parameters(), lr=5e-4, eps=1e-7)

# Training
data_folder = f'../data/{property_name}'
train_model(property_name, data_folder, optimizer, max_train_steps=2400)  # Ideal: 1000-2400 steps

# Save final model
save_model(f'saved_models/textual_inversion/{property_name}', f'{property_name}_final.pt')

***** Running Training *****
  Num. Examples = 600
  Num. Epochs = 9
loss: 0.0765, lr: 0.0005, epoch: 1/9, step: 1/600
loss: 0.468, lr: 0.0005, epoch: 1/9, step: 2/600
loss: 0.036, lr: 0.0005, epoch: 1/9, step: 3/600
loss: 0.2474, lr: 0.0005, epoch: 1/9, step: 4/600
loss: 0.0977, lr: 0.0005, epoch: 1/9, step: 5/600
loss: 0.0037, lr: 0.0005, epoch: 1/9, step: 6/600
loss: 0.0031, lr: 0.0005, epoch: 1/9, step: 7/600
loss: 0.0421, lr: 0.0005, epoch: 1/9, step: 8/600
loss: 0.0057, lr: 0.0005, epoch: 1/9, step: 9/600
loss: 0.0034, lr: 0.0005, epoch: 1/9, step: 10/600
loss: 0.0032, lr: 0.0005, epoch: 1/9, step: 11/600
loss: 0.0157, lr: 0.0005, epoch: 1/9, step: 12/600
loss: 0.2542, lr: 0.0005, epoch: 1/9, step: 13/600
loss: 0.8105, lr: 0.0005, epoch: 1/9, step: 14/600
loss: 0.0097, lr: 0.0005, epoch: 1/9, step: 15/600
loss: 0.5576, lr: 0.0005, epoch: 1/9, step: 16/600
loss: 0.1816, lr: 0.0005, epoch: 1/9, step: 17/600
loss: 0.0666, lr: 0.0005, epoch: 1/9, step: 18/600
loss: 0.2512, lr: 0.0005